<a href="https://colab.research.google.com/github/LC1332/Haruhi-2-Dev/blob/main/notebook/%E4%BD%BF%E7%94%A8ChatHaruhi54K%E7%94%9F%E6%88%90%E6%95%B0%E6%8D%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] RoleLLM的数据
- [ ] 其他中文的hf的一些数据
- [ ] ChatHaruhi54K本身的数据

这里我们假设输入数据已经组织成了hf格式

可以有效载入到chatbot中

In [ ]:
!pip -q install openai tiktoken langchain chromadb datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

组织数据

In [ ]:
from datasets import load_dataset

dataset = load_dataset("silk-road/ChatHaruhi-54K-Role-Playing-Dialogue")

role2tuple = {}

from tqdm import tqdm
for data in tqdm( dataset['train'] ):
    agent_role = data['agent_role']
    if agent_role not in role2tuple:
        role2tuple[agent_role] = []

    user_role = data['user_role']
    user_question = data['user_question']

    query = user_role + ":" + user_question + ""

    agent_response = data['agent_response']

    target = agent_role + ":" + agent_response + ""

    # role2tuple[agent_role].append((query, target))

    history_str = ""

    tmp_data = {
        "query": query,
        "target": target,
        "history_str": history_str,
    }

    role2tuple[agent_role].append(tmp_data)

    more_dialogues = data['more_dialogues']

    history_str += query + "\n" + target

    history_prefix = history_str

    if len(more_dialogues) > 0:
        n = len( more_dialogues )
        for i in range(n):
            if i == 0:
                continue

            sent = more_dialogues[i]
            if len(sent) < len(agent_role):
                continue

            if sent.startswith(agent_role):
                query = more_dialogues[i-1]
                target = sent

                history_str = history_prefix
                for j in range(i-1):
                    history_str += "\n" + more_dialogues[j]

                tmp_data = {
                    "query": query,
                    "target": target,
                    "history_str": history_str,
                }

                role2tuple[agent_role].append(tmp_data)

                # print(tuple_data)

                # break

        # break



100%|██████████| 54726/54726 [00:04<00:00, 13667.94it/s]


tuple数据会保留在role2tuple

In [ ]:
print(role2tuple['Harry'][100])

{'query': 'Voldemort:「You think you know more magic than I do? Than I, than Lord Voldemort, who has performed magic that Dumbledore himself never dreamed of?」', 'target': 'Harry:「Oh, he dreamed of it, but he knew more than you, knew enough not to do what you’ve done.」', 'history_str': ''}


训练的参数设定

In [ ]:
K_SEARCH = 5
MAX_LEN_STORY = 1000 #这个是按照token算的
MAX_LEN_HISTORY = 1200 # count with token

In [ ]:
import os
import openai
key = "key is not necessary here"
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

In [ ]:

%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev

/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 916, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 916 (delta 150), reused 157 (delta 107), pack-reused 693
Receiving objects: 100% (916/916), 105.96 MiB | 40.31 MiB/s, done.
Resolving deltas: 100% (480/480), done.
/content/Haruhi-2-Dev


In [ ]:
!ls

ChatHaruhi  data  LICENSE  notebook  Readme.md


In [ ]:
from ChatHaruhi import ChatHaruhi

from ChatHaruhi.ChromaDB import ChromaDB

class ChatHaruhiTrain(ChatHaruhi):

    def build_story_db_from_vec( self, texts, vecs ):
        self.db = ChromaDB()
        self.stories = texts
        self.db.init_from_docs( vecs, texts)

    def add_story_with_expire(self, query, expire_story):
        if self.db is None:
            print("No vec DB！")
            return

        query_vec = self.embedding(query)
        stories = self.db.search(query_vec, self.k_search)

        story_string = self.story_prefix_prompt + self.dialogue_divide_token

        sum_story_token = self.tokenizer(story_string)

        for story in stories:
            if expire_story is not None and expire_story.strip() == story.strip():
                continue

            if expire_story == None and query.strip() in story.strip():
                continue

            story_token = self.tokenizer(story.strip()) + self.tokenizer(self.dialogue_divide_token)
            if sum_story_token + story_token > self.max_len_story:
                break
            else:
                sum_story_token += story_token
                story_string += story.strip() + self.dialogue_divide_token

        self.llm.user_message(story_string)

    def generate_prompt(self, query, history_str, expire_story ):
        # 这里修改下其他超参，不规范删了
        # self.k_search = 5
        # self.max_len_story = 1500
        # self.max_len_history = 1200
        self.story_prefix_prompt = "\nClassic scenes for the role are as follows:"

        self.llm.initialize_message()

        self.llm.system_message(self.system_prompt)

        self.add_story_with_expire(query, expire_story)

        # self.add_history(history)
        history_message = self.dialogue_divide_token + history_str.strip()
        self.llm.user_message(history)

        self.llm.user_message(query)

        # self.llm.user_message(target)

        return self.llm.messages

    def add_history(self, history_list):

        if len(history_list) == 0:
            return

        sum_history_token = 0
        flag = 0
        for history in history_list:
            current_count = 0
            if history is not None:
                current_count += self.tokenizer(history)

            sum_history_token += current_count
            if sum_history_token > self.max_len_history:
                break
            else:
                flag += 1

        if flag == 0:
            print('warning! no history added. the last dialogue is too long.')

        # 是否添加历史前缀，
        history_message = ""
        for history in history_list[-flag:]:
            history_message += history
        self.llm.user_message(history_message)

这个时候我们一次性把所有的RoleLLM都先申请出来

In [ ]:
role_name_Haruhiu = {'汤师爷': 'tangshiye', 'tangshiye': 'tangshiye', 'Tangshiye': 'tangshiye',
                     '慕容复': 'murongfu', 'murongfu': 'murongfu', 'Murongfu': 'murongfu',
                     '李云龙': 'liyunlong', 'liyunlong': 'liyunlong', 'Liyunlong': 'liyunlong',
                     'Luna': 'Luna', '王多鱼': 'wangduoyu', 'wangduoyu': 'wangduoyu',
                     'Wangduoyu': 'wangduoyu', 'Ron': 'Ron', '鸠摩智': 'jiumozhi',
                     'jiumozhi': 'jiumozhi', 'Jiumozhi': 'jiumozhi', 'Snape': 'Snape',
                     '凉宫春日': 'haruhi', 'haruhi': 'haruhi', 'Haruhi': 'haruhi',
                     'Malfoy': 'Malfoy', '虚竹': 'xuzhu', 'xuzhu': 'xuzhu',
                     'Xuzhu': 'xuzhu', '萧峰': 'xiaofeng',
                     'xiaofeng': 'xiaofeng', 'Xiaofeng': 'xiaofeng', '段誉': 'duanyu',
                     'duanyu': 'duanyu', 'Duanyu': 'duanyu', 'Hermione': 'Hermione',
                     'Dumbledore': 'Dumbledore', '王语嫣': 'wangyuyan', 'wangyuyan':
                     'wangyuyan', 'Wangyuyan': 'wangyuyan', 'Harry': 'Harry',
                     'McGonagall': 'McGonagall', '白展堂': 'baizhantang',
                     'baizhantang': 'baizhantang', 'Baizhantang': 'baizhantang',
                     '佟湘玉': 'tongxiangyu', 'tongxiangyu': 'tongxiangyu',
                     'Tongxiangyu': 'tongxiangyu', '郭芙蓉': 'guofurong',
                     'guofurong': 'guofurong', 'Guofurong': 'guofurong', '流浪者': 'wanderer',
                     'wanderer': 'wanderer', 'Wanderer': 'wanderer', '钟离': 'zhongli',
                     'zhongli': 'zhongli', 'Zhongli': 'zhongli', '胡桃': 'hutao', 'hutao': 'hutao',
                     'Hutao': 'hutao', 'Sheldon': 'Sheldon', 'Raj': 'Raj',
                     'Penny': 'Penny', '韦小宝': 'weixiaobao', 'weixiaobao': 'weixiaobao',
                     'Weixiaobao': 'weixiaobao', '乔峰': 'qiaofeng', 'qiaofeng': 'qiaofeng',
                     'Qiaofeng': 'qiaofeng', '神里绫华': 'ayaka', 'ayaka': 'ayaka',
                     'Ayaka': 'ayaka', '雷电将军': 'raidenShogun', 'raidenShogun': 'raidenShogun',
                     'RaidenShogun': 'raidenShogun', '于谦': 'yuqian', 'yuqian': 'yuqian',
                     'Yuqian': 'yuqian', 'Professor McGonagall': 'McGonagall',
                     'Professor Dumbledore': 'Dumbledore'}

In [ ]:
count = 0

for role in role2tuple:
    if role in role_name_Haruhiu:
        for ele in role2tuple[role]:
            if isinstance( ele, dict):
                count += 1

print(count)

62663


新建各自的chatbot

In [ ]:
import zipfile
import os

try:
    os.makedirs("characters_zip")
except:
    pass
try:
    os.makedirs("characters")
except:
    pass

role_en2bots = {}


from tqdm import tqdm

for ai_role_en in tqdm( role_name_Haruhiu.values() ):
    if ai_role_en in role_en2bots:
        continue

    role_en2bots[ai_role_en] = ChatHaruhiTrain(role_name = ai_role_en)

100%|██████████| 76/76 [00:05<00:00, 14.27it/s]


对RoleLLM的数据进行组织

In [ ]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
print(dir(chatbot.db))

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', 'client', 'collection', 'init_db', 'init_from_docs', 'load', 'path', 'save', 'search']


'add', 'construct', 'copy', 'count', 'database', 'delete', 'dict', 'from_orm', 'get', 'id', 'json', 'metadata', 'modify', 'name', 'parse_file', 'parse_obj', 'parse_raw', 'peek', 'query', 'schema', 'schema_json', 'tenant', 'update', 'update_forward_refs', 'upsert', 'validate']
[21]
0s


In [ ]:
print(chatbot.db.collection.count())

1048


In [ ]:
result = chatbot.db.collection.get()
print(result.keys())
print(len(result['documents']))

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])
1048


In [ ]:
from datasets import load_dataset
from tqdm import tqdm

save_datas = []

# for role_name in tqdm(role_from_roleLLM):

for role_name_zh in role2tuple.keys():
    if role_name_zh not in role_name_Haruhiu:
        continue
    role_name_en = role_name_Haruhiu[role_name_zh]

    chatbot = role_en2bots[role_name_en]


    chatbot.k_search = K_SEARCH
    chatbot.max_len_story = MAX_LEN_STORY
    chatbot.max_len_history = MAX_LEN_HISTORY

    all_tuples = role2tuple[role_name_zh]

    # break

    for tuple_data in tqdm(all_tuples):
        query = tuple_data['query']
        target = tuple_data['target']
        history = tuple_data['history_str']
        messages = chatbot.generate_prompt(query, history, None)
        # print(prompt)

        prompt = ""

        for msg in messages:
            if isinstance(msg, HumanMessage):
                prompt += msg.content + "\n"
            elif isinstance(msg, AIMessage):
                prompt += msg.content + "\n"
            elif isinstance(msg, SystemMessage):
                prompt += msg.content + "\n"

        save_data = {
            'context': prompt,
            'target': target
        }

        save_datas.append(save_data)

        # break

    # break

100%|██████████| 1816/1816 [03:06<00:00,  9.73it/s]


In [ ]:
print(all_tuples[2].keys())

dict_keys(['query', 'target', 'history_str'])


In [ ]:
print(all_tuples[3])

{'query': '陈近南:「大家是好朋友，这事虽然干系不小，却也不能相瞒。混在宫里当小太监的，就是我那小徒韦小宝自己。小宝，你出来见过众位前辈。」', 'target': '韦小宝:「哈哈哈，原来是众位前辈啊！小宝在宫里混得风生水起，怎么会忘记了这帮好兄弟呢？来，来，让我好好瞧瞧你们。」', 'history_str': ''}


In [ ]:
print(save_data['context'])

print()

print(save_data['target'])

I want you to act like 韦小宝 from 鹿鼎记.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 韦小宝 using the tone, manner and vocabulary 韦小宝 would use. 
You must know all of the knowledge of 韦小宝.
注意韦小宝是狡诈、机智、善于应变的
对话中常常不拘礼节，不怕得罪他人，有时甚至带有一些粗鲁和挑衅
常常以发财为目标，利用各种机会来获取财富。


Classic scenes for the role are as follows:
###
李自成:「大丈夫恩怨分明，那日你师父没杀我，今日我也饶你一命。自今而后，你再向我女儿看上一眼、说一句话，我把你全身砸成了肉酱」
韦小宝:「大丈夫一言既出，那就怎样。那日在三圣庵里，你和你的姘头陈圆圆，已将阿珂许配我为妻，难道又想赖么。你不许我向自己老婆看上一眼，说一句话，天下哪有这样的岳父大人」
阿珂:「爹，咱们走，别理这小子胡说八道。他狗嘴里长不出象牙，有甚么好话说了」
韦小宝:「好啊，你终于认了他啦。这父母之命，你听是不听」
###
阿珂:「甚么你说我娘有危险么」
韦小宝:「你娘倒没危险，我却有大大的危险」
阿珂:「怎么危险到你身上了」
韦小宝:「胡大哥跟我八拜之交，是结义兄弟。倘若他在兵荒马乱之中，却跟你娘搂搂抱抱，勾勾搭搭，可不是做了我的岳父吗这辈份是一塌胡涂了」
###
阿珂:「不不要郑郑公子是你么」
旁白:韦小宝想"你做梦也梦到郑公子，只道是他爬上了你床，好快活么"
韦小宝:「是我」
阿珂:「不，不你不要」
郑克爽:「阿珂，阿珂，你在哪里」
阿珂:「你你是谁怎么我我」
韦小宝:「是你的亲老公，你也听不出」
阿珂:「我在这里放开手小鬼，你干干甚么」
郑克爽:「甚么」
韦小宝:「好师弟，求求你，快放开我」
韦小宝:「我在床上，抱着我老婆。我在洞房花烛，你来干甚么要闹新房么」
###
阿珂:「你你怎么不早说他又说什么」
韦小宝:「他说，这几位侠女要到台湾去玩玩，他就带她们同

save_datas是一个list of dict
save_name = "/content/ChatHaruhi_rolellm_fusion.jsonl"

帮我用utf-8编码，jsonl格式，保存save_datas中的数据
ensure_ascii = False

In [ ]:
print(len(save_datas))

13166


In [ ]:
import json

save_name = "/content/ChatHaruhi_54K_retide.jsonl"

with open(save_name, 'w', encoding='utf8') as f:
    for data in save_datas:
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')